In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import *
import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, "/home/peisenha/structExpectations/development/estimations/estimation-current")
os.chdir("/home/peisenha/structExpectations/development/estimations/estimation-current")
import yaml

In [2]:

yaml.load(open("resources/model_spec_init_rev.yml"), Loader=yaml.Loader)["EXOG_PROC"]["child_age_max"]
yaml.load(open("resources/model_spec_init.yml"), Loader=yaml.Loader)["EXOG_PROC"]["child_age_max"]

18

In [3]:
model_params_start = pd.read_pickle("start.soepy.pkl")

In [4]:
df_sim_base = soepy.simulate(model_params_start, "resources/model_spec_init.yml")
df_sim_base = df_alignment(df_sim_base)

In [5]:
df_sim_rev = soepy.simulate(model_params_start, "resources/model_spec_init_rev.yml")
df_sim_rev = df_alignment(df_sim_rev)

In [6]:
df_sim_base.head(3)

Education_Level  Lagged_Choice  Experience_Part_Time  \
Identifier Period                                                        
0          0               Medium              0                     0   
           1               Medium              2                     0   
           2               Medium              2                     0   

                   Experience_Full_Time  Type  Age_Youngest_Child  \
Identifier Period                                                   
0          0                          0     1                  -1   
           1                          1     1                  -1   
           2                          2     1                  -1   

                   Partner_Indicator Choice  Log_Systematic_Wage  \
Identifier Period                                                  
0          0                       0   Full             2.343967   
           1                       0   Full             2.426523   
           2                       0   Full             2.475118   

                   Period_Wage_P  ...  Flow_Utility_N  Flow_Utility_P  \
Identifier Period                 ...                                   
0          0           14.235549  ...       -0.101138       -0.043553   
           1           12.158107  ...       -0.088089       -0.047413   
           2           11.097556  ...       -0.085855       -0.045581   

                   Flow_Utility_F  Continuation_Value_N  Continuation_Value_P  \
Identifier Period                                                               
0          0            -0.026275             -1.332090             -1.327753   
           1            -0.028462             -1.311863             -1.308262   
           2            -0.034307             -1.288663             -1.285392   

                   Continuation_Value_F  Value_Function_N  Value_Function_P  \
Identifier Period                                                             
0          0                  -1.324500         -1.406586         -1.344750   
           1                  -1.305608         -1.373716         -1.329510   
           2                  -1.283150         -1.348745         -1.305265   

                   Value_Function_F  Wage_Observed  
Identifier Period                                   
0          0              -1.324285      19.106368  
           1              -1.307958      16.251337  
           2              -1.291794      11.271681  

[3 rows x 24 columns]

In [28]:
rslt_base = df_sim_base[df_sim_base["Education_Level"] == "High"].groupby("Period")["Choice"].value_counts(normalize=True)

In [29]:
rslt_rev = df_sim_rev[df_sim_rev["Education_Level"] == "High"].groupby("Period")["Choice"].value_counts(normalize=True) 

In [30]:
columns = ["value", "upper", "lower"]

model_params_start.loc[("const_wage_eq"), "value"] = 2.2

model_params_start.loc[("exp_returns", "gamma_1s1"), "value"] = 0.125
model_params_start.loc[("exp_returns", "gamma_1s2"), "value"] = 0.20
model_params_start.loc[("exp_returns", "gamma_1s3"), "value"] = 0.30


model_params_start.loc[("disutil_work", "child_02_f"), columns] = [+0.282430, +1.0, +0.0]
model_params_start.loc[("disutil_work", "child_02_p"), columns] = [-0.210744, +0.0, -1.0]

model_params_start.loc[("disutil_work", "child_35_f"), columns] = [+0.188765, +1.0, +0.0]
model_params_start.loc[("disutil_work", "child_35_p"), columns] = [-0.200325, +0.0, -1.0]

model_params_start.loc[("disutil_work", "child_6orolder_f"), columns] = [+0.238225, +1.0, +0.0]
model_params_start.loc[("disutil_work", "child_6orolder_p"), columns] = [-0.408204, +0.0, -1.0]

model_params_start.loc[("disutil_work", "no_kids_f_educ_high"), columns]   = [+0.256306, +1.0, +0.0]
model_params_start.loc[("disutil_work", "no_kids_f_educ_low"), columns]    = [+0.240291, +1.0, +0.0]
model_params_start.loc[("disutil_work", "no_kids_f_educ_middle"), columns] = [+0.056999, +1.0, +0.0]

model_params_start.loc[("disutil_work", "no_kids_p_educ_high"), columns]   = [-0.472124, +0.0, -1.0]
model_params_start.loc[("disutil_work", "no_kids_p_educ_low"), columns]    = [-0.595835, +0.0, -1.0]
model_params_start.loc[("disutil_work", "no_kids_p_educ_middle"), columns] = [-0.484731, +0.0, -1.0]

model_params_start.loc[("disutil_work", "yes_kids_f_educ_high"), columns]   = [+0.799036, +1.5, -0.0]
model_params_start.loc[("disutil_work", "yes_kids_f_educ_low"), columns]    = [+1.257840, +1.5, -0.0]
model_params_start.loc[("disutil_work", "yes_kids_f_educ_middle"), columns] = [+0.991504, +1.5, -0.0]

model_params_start.loc[("disutil_work", "yes_kids_p_educ_high"), columns]   = [-0.396360, -0.0, -1.0]
model_params_start.loc[("disutil_work", "yes_kids_p_educ_low"), columns]    = [-0.432508, -0.0, -1.0]
model_params_start.loc[("disutil_work", "yes_kids_p_educ_middle"), columns] = [-0.351832, -0.0, -1.1]

model_params_start.to_pickle("start.soepy.pkl")

Period  Choice
0       Full      0.000000
        Home     -0.000109
        Part      0.000109
1       Full      0.000000
        Home     -0.000109
        Part      0.000109
2       Full      0.000000
        Home     -0.000218
        Part      0.000218
3       Full      0.000218
        Home     -0.000326
        Part      0.000109
4       Full      0.000000
        Home     -0.000109
        Part      0.000109
5       Full      0.000326
        Home     -0.000326
        Part      0.000000
6       Full      0.000109
        Home     -0.000109
        Part      0.000000
7       Full      0.000218
        Home     -0.000218
        Part      0.000000
8       Full      0.000218
        Home     -0.000653
        Part      0.000435
9       Full      0.000109
        Home      0.000000
        Part     -0.000109
10      Full      0.000109
        Home     -0.000762
        Part      0.000653
11      Full      0.000326
        Home     -0.000544
        Part      0.000218
12      Full 